# NETFLIX Recommandation System

The Dataset was collected on Kaggle. https://www.kaggle.com/shivamb/netflix-shows

TV Shows and Movies listed on Netflix

This dataset consists of tv shows and movies available on Netflix as of 2019. The dataset is collected from Flixable which is a third-party Netflix search engine.

